In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install neattext

     |████████████████████████████████| 112 kB 7.8 MB/s 


In [3]:
import pandas as pd
import string #Opérations courantes sur les chaînes
import re   # pour utiliser  des expressions régulières ex:$
import nltk #boites outils ntk
from sklearn.model_selection import train_test_split
import neattext as nt
import neattext.functions as nfx

In [4]:
#Cleaning & Preprocessing
def process(text):
# la normalisation
    regex = re.compile(r'[إأٱآا]') #sauvegarder l'expression réguliere (r=expression reguliere)
    text = re.sub(regex, 'ا', str(text)) #remblacer des expressions par d'autres dans varia texte
    regex = re.compile(r'[ى]')
    text = re.sub(regex, 'ي', str(text))
    regex = re.compile(r'[ؤئ]')
    text = re.sub(regex, 'ء', str(text))
    text = re.sub('\n', ' ', str(text))
    text = re.sub(' و ', ' ', str(text))
    text = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    text = text.rstrip()
#supprimer les diacritiques
    arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)   #verbose: séparer visuellement les expre et ajouter des commentaires
    text = re.sub(arabic_diacritics, '', text)
    

#supprimer les ponctuations
    #liste des ponctuations arabe et anglais qu'on veut supprimer
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''    # utiliser ''' :multiples caractére de chaine
    english_punctuations = string.punctuation              #(!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)
    punctuations_list = arabic_punctuations + english_punctuations
    #table de mappage
    translator = str.maketrans('','', punctuations_list) # le 1er elem a remplacer/le 2eme le remplacement,3eme=supprimer
    text= text.translate(translator) #chaine ou les caractere sont remplacé par d'autres dns table de mappage

#supprimer les chaines répétés
    text=re.sub(r'(.)\1+', r'\1', str(text)) 

#supprimer les mots vides
    stopwords=nltk.corpus.stopwords.words('arabic') #les mots vides en arabe
    txt=[word for word in text if word not in stopwords]

#supprimer les émojis
    myre = re.compile(u'['
    u'\U0001F300-\U0001F64F'
    u'\U0001F680-\U0001F6FF'     
    u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    text=myre.sub('', text)
    
    
    return text

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# WORD2VEC

# Preparation du model

In [7]:
import spacy

In [8]:
# load AraVec Spacy model
%cd /content/drive/MyDrive/Word2Vec
nlp = spacy.load("spacy.aravec.model")
%cd ..
%cd ..
%cd ..

/content/drive/MyDrive/Word2Vec
/content/drive/My Drive
/content/drive
/content


In [9]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = process(text)
        return self.tokenizer(preprocessed)

In [10]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

In [11]:
#retourne le vecteur correspondant a la phrase x
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

# Machine Learning

In [12]:
import joblib

In [13]:
%cd /content/drive/MyDrive
rf=joblib.load("RF.hdf5")
%cd ..
%cd ..

/content/drive/MyDrive
/content/drive
/content


In [14]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [15]:
emotion={0:'none', 1:'anger', 2:'sadness', 3:'joy', 4:'surprise', 5:'love', 6:'sympathy', 7:'fear'}

In [16]:
phrase = "الطعام سيء"
phrase = pd.Series(phrase)

In [17]:
# Get the sentence embeddings for the train dataset
vec_text_train = phrase.apply(lambda x: get_vec(x))
vec_text_train = vec_text_train.to_numpy()
vec_text_train = vec_text_train.reshape(-1, 1)
vec_text_train = np.concatenate(np.concatenate(vec_text_train, axis = 0), axis = 0).reshape(-1, 300)

In [18]:
pred=rf.predict(vec_text_train)

In [19]:
emotion[pred[0]]

'sadness'

In [ ]:
#-------------------------------

In [20]:
phrase = "الطعام لذيذ جدا"
phrase = pd.Series(phrase)

In [21]:
# Get the sentence embeddings for the train dataset
vec_text_train = phrase.apply(lambda x: get_vec(x))
vec_text_train = vec_text_train.to_numpy()
vec_text_train = vec_text_train.reshape(-1, 1)
vec_text_train = np.concatenate(np.concatenate(vec_text_train, axis = 0), axis = 0).reshape(-1, 300)

In [22]:
pred=rf.predict(vec_text_train)

In [23]:
emotion[pred[0]]

'joy'

In [ ]:
#-------------------------------

In [ ]:
phrase = "الطعام لذيذ جدا"
phrase = pd.Series(phrase)

In [ ]:
# Get the sentence embeddings for the train dataset
vec_text_train = phrase.apply(lambda x: get_vec(x))
vec_text_train = vec_text_train.to_numpy()
vec_text_train = vec_text_train.reshape(-1, 1)
vec_text_train = np.concatenate(np.concatenate(vec_text_train, axis = 0), axis = 0).reshape(-1, 300)

In [ ]:
pred=rf.predict(vec_text_train)

In [ ]:
emotion[pred[0]]

In [ ]:
#------------------------------

In [ ]:
phrase = "كدت أموت من الخوف"
phrase = pd.Series(phrase)

In [ ]:
# Get the sentence embeddings for the train dataset
vec_text_train = phrase.apply(lambda x: get_vec(x))
vec_text_train = vec_text_train.to_numpy()
vec_text_train = vec_text_train.reshape(-1, 1)
vec_text_train = np.concatenate(np.concatenate(vec_text_train, axis = 0), axis = 0).reshape(-1, 300)

In [ ]:
pred=rf.predict(vec_text_train)

In [ ]:
emotion[pred[0]]

'fear'